# Obtener datos con web scraping y una BD local

## SQL

Para conectarme a la base de datos de mi computadora

In [1]:
password="root"

In [2]:
import mysql.connector as mariadb
import sys


In [3]:
try:
    conn = mariadb.connect(
      user="root",
      password=password,
      host="127.0.0.1",
      port=3306
      )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)


Obtengo el cursor que me permite ejecutar comandos SQL en mi computadora

In [4]:
cursor=conn.cursor()


Esto sólo es necesario cuando no tengo la BD

In [5]:
try:
    cursor.execute("CREATE DATABASE noticias")
except:
    pass

Así creé la tabla donde guardé mis resultados. Usé el título como llave primaria.

In [6]:

notas="""CREATE TABLE notas(
                titulo VARCHAR (500) NOT NULL,
                resumen VARCHAR (700) NOT NULL,
                autor VARCHAR (100) NOT NULL,
                periodico VARCHAR (100),
                noticia VARCHAR (5000),
                fecha VARCHAR (100),
                lugar VARCHAR (100),
                PRIMARY KEY (titulo)
            
)
    """

In [7]:
cursor.execute("USE %s"%"noticias")
try:
    cursor.execute(notas)
except:
    pass

Esta es una función para insertar todos los datos que vaya a scrapear a mi BD

In [8]:
def insert(d):
    
    keys=[]
    values=[]
    for k,v in d.items():
        keys.append(k)
        values.append(v)
        
    names=keys
    for v in values:
        names.append(v)
    if not d["noticia"]:
        return
    s="""INSERT INTO notas (%s, %s, %s, %s, %s, %s, %s) VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s')"""%tuple(names)
    #Los duplicados no los insertamos
    try:
        cursor.execute(s)
    except:
        print("Duplicado evitado...")
    
    

## Scrapeo!

Estas son algunas librerías que usé. Encontré que newspaper hacía lo que quería por mí, pero como los resultados no siempre eran los esperados, decidí bajar la información con BS4 y sólo sacar los links de las noticias con apoyo de esta librería newspaper

In [9]:
#Para hacer scraping manual
from bs4 import BeautifulSoup

#librerías de scraping de python!
import requests 
from newspaper import Article
import newspaper

#Para guardar la fecha en que obtuve los datos
from datetime import date

#Para hacer peticiones cada cierto tiempo y evitar que me baneen.
import time

Esta función me permite tomar los tags de cada periódico (que obtuve manualmente analizando la estructura de la página) y obtener los datos que la página tiene disponibles. Como podemos ver, no siempre se obtienen los datos que se espera obtener. Cuando eso pasa, lo especificamos.

In [10]:
def scrape(url,periodico):
    #variables
    titulo,resumen,autor,fecha,lugar,texto,fuente=periodico
    nota={}
    #scraping!
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'html.parser')
    
    #Para cada tag, vemos si está en el html que descargamos
    try:
        nota["titulo"] = soup.select_one(titulo).get_text().strip()[:100]
    except:
        nota["titulo"]= "Sin titulo"
    nota["periodico"]=fuente[:100]
    try:
        nota["resumen"] = soup.select_one(resumen).get_text().strip()[:700]
    except:
        nota["resumen"]="Sin Resumen"
    try:
        nota["autor"] = soup.select_one(autor).get_text().strip()[:100]
    except:
        nota["autor"]="Anonimo"
    nota["fecha"] = date.today().strftime("%d/%m/%Y")
    try:
        nota["lugar"] = soup.select_one(lugar).get_text().strip()[:100]
    except:
        nota["lugar"]="Sin Lugar"
    try:
        nota["noticia"] = soup.select_one(texto).get_text().strip()[:5000].replace("''","")
    except:
        nota["noticia"] = 0
        
    #Regresamos un diccionario con toda la info
    return nota
    
    

Estos son los tags por periódico que le vamos a pasar a la función

In [11]:
milenio=["h1.title","h2.summary","span.author","time","span.location",'div#content-body.media-container.news',"Milenio"]

el_universal=["h1.h1","h2.h2","span.ce12-DatosArticulo_autor","span.ce12-DatosArticulo_ElementoFecha",
              "none","div.field.field-name-body.field-type-text-with-summary.field-label-hidden",
             "El Universal"]

la_jornada=["h2#article-title-tts.title.title-default","None","a.author","span.time-pill","em",
            "div#article-content-tts.article-content.ljn-nota-contenido","La Jornada"]


In [12]:
periodicos=[["https://www.milenio.com",milenio],["https://www.eluniversal.com.mx",el_universal],
            ["https://www.jornada.com.mx",la_jornada]]


Ahora sólo obtenemos las noticias del día con los urls que se obtienen de la página principal. Después añadimos la noticia a una lista de noticias. Al final insertamos todas las noticias que encontramos a la base de datos.

In [13]:
for periodico in periodicos:
    #obtenemos url de las noticias de hoy!
    site = newspaper.build(periodico[0])
    urls = site.article_urls()
    #agregamos las noticias aquí
    noticias = []
    i=0
    
    print(periodico)
    for url in urls:
        i+=1
        #si algo sale mal en la conexión es mejor no seguir intentando para que no nos baneen,
        # pasamos al siguiente periodico
        try:
            noticias.append(scrape(url,periodico[1])) 
        except:
            break
        #esperamos un segundo para que no nos baneen por saturar el servidor
        #(De todas formas es poco probable saturarlo porque son como 200 noticias por periódico)
        time.sleep(1)
        
        if not (i%20):
            print(i,"notas bajadas")
            
    print("Insertando noticias")
    for noticia in noticias:
        insert(noticia)

['https://www.milenio.com', ['h1.title', 'h2.summary', 'span.author', 'time', 'span.location', 'div#content-body.media-container.news', 'Milenio']]
20 notas bajadas
40 notas bajadas
60 notas bajadas
80 notas bajadas
100 notas bajadas
120 notas bajadas
140 notas bajadas
160 notas bajadas
Insertando noticias
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...
Duplicado evitado...

In [14]:
conn.commit()

In [15]:
cursor.execute("select * from notas where fecha = fecha")

In [16]:
len(cursor.fetchall())

2506